<a href="https://colab.research.google.com/github/irwanmaulana42/google-collab/blob/master/Iconnet_Linear_Regresi_dan_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Perbandingan Regresi Linear dan Random Forest pada wilayah Jabodetabek dan Banten


In [1]:
import pandas as pd


In [28]:

# https://github.com/irwanmaulana42/google-collab/raw/refs/heads/master/data_penjualan_iconnet_2022_2023.xlsx


df=pd.read_excel('https://github.com/irwanmaulana42/google-collab/raw/refs/heads/master/data_penjualan_iconnet_2022_2023.xlsx')
df.head()


,BW,MONTH,YEAR,WILAYAH,latitude,longitude
0,10 Mbps,April,2022,Kab. Bekasi,-6.154664,107.0235344
1,20 Mbps,April,2022,Kab. Bekasi,-6.156413,107.028303
2,10 Mbps,April,2022,Kab. Bekasi,-6.156794,107.02828844597303
3,10 Mbps,April,2022,Kab. Bekasi,-6.156961,107.02855847776
4,20 Mbps,April,2022,Kab. Bekasi,-6.158325,107.019362


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101325 entries, 0 to 101324
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   BW         101325 non-null  object 
 1   MONTH      101325 non-null  object 
 2   YEAR       101325 non-null  int64  
 3   WILAYAH    100350 non-null  object 
 4   latitude   101207 non-null  float64
 5   longitude  101275 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.6+ MB


In [30]:
# prompt: delete latitude and longitude on table

if 'latitude' in df.columns:
  df = df.drop('latitude', axis=1)
if 'longitude' in df.columns:
  df = df.drop('longitude', axis=1)

df.head()


,BW,MONTH,YEAR,WILAYAH
0,10 Mbps,April,2022,Kab. Bekasi
1,20 Mbps,April,2022,Kab. Bekasi
2,10 Mbps,April,2022,Kab. Bekasi
3,10 Mbps,April,2022,Kab. Bekasi
4,20 Mbps,April,2022,Kab. Bekasi


In [31]:
# prompt: clean data BW from 20 Mbps or somehting, to integer

import pandas as pd

# Assuming 'BW' is the column name containing bandwidth information
# You may need to adjust the column name if it's different

# Ubah semua nilai di kolom 'BW' menjadi string terlebih dahulu
df['BW'] = df['BW'].astype(str)

# Remove non-numeric characters and convert to numeric, replacing errors with NaN
df['BW'] = pd.to_numeric(df['BW'].str.replace(r'[^0-9]', '', regex=True), errors='coerce')

# Fill NaN values with 0, you might want to choose a different strategy based on your data
df['BW'] = df['BW'].fillna(0).astype(int)

print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101325 entries, 0 to 101324
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   BW       101325 non-null  int64 
 1   MONTH    101325 non-null  object
 2   YEAR     101325 non-null  int64 
 3   WILAYAH  100350 non-null  object
dtypes: int64(2), object(2)
memory usage: 3.1+ MB
None
   BW  MONTH  YEAR      WILAYAH
0  10  April  2022  Kab. Bekasi
1  20  April  2022  Kab. Bekasi
2  10  April  2022  Kab. Bekasi
3  10  April  2022  Kab. Bekasi
4  20  April  2022  Kab. Bekasi


In [32]:
# prompt: get grouping wilayah from table

wilayah_counts = df.groupby('WILAYAH')['WILAYAH'].count()
print(wilayah_counts)
print(df.head())

WILAYAH
Kab. Bekasi                    3664
Kab. Bogor                    16574
Kab. Lebak                     1223
Kab. Pandeglang                3298
Kab. Serang                     305
Kab. Tangerang                 8300
Kota Adm. Jakarta  Selatan        1
Kota Adm. Jakarta Barat        1792
Kota Adm. Jakarta Pusat        1384
Kota Adm. Jakarta Selatan      6348
Kota Adm. Jakarta Timur        5995
Kota Adm. Jakarta Utara        1994
Kota Bekasi                    1586
Kota Bogor                      722
Kota Cilegon                   2085
Kota Depok                    21057
Kota Rangkasbitung              878
Kota Serang                    6198
Kota Tangerang                 8239
Kota Tangerang Selatan         8707
Name: WILAYAH, dtype: int64
   BW  MONTH  YEAR      WILAYAH
0  10  April  2022  Kab. Bekasi
1  20  April  2022  Kab. Bekasi
2  10  April  2022  Kab. Bekasi
3  10  April  2022  Kab. Bekasi
4  20  April  2022  Kab. Bekasi


In [36]:
# Fungsi untuk menetapkan kode wilayah
def kode_wilayah(wilayah):
    # Cek apakah wilayah adalah string, jika tidak, kembalikan None
    if isinstance(wilayah, str):
        if 'Jakarta' in wilayah:
            return 1
        elif 'Bogor' in wilayah:
            return 2
        elif 'Depok' in wilayah:
            return 3
        elif 'Tangerang' in wilayah:
            return 4
        elif 'Bekasi' in wilayah:
            return 5
        elif 'Serang' in wilayah or 'Cilegon' in wilayah or 'Lebak' in wilayah or 'Pandeglang' in wilayah:
            return 6
        else:
            return None
    else:
        return None

# Menambahkan kolom 'Kode Wilayah'
df['Kode Wilayah'] = df['WILAYAH'].apply(kode_wilayah)

# Menghapus jika Kode wilayah NA atau NaN
df = df.dropna(subset=['Kode Wilayah'])

# Mengubah kode wilayah menjadi integer
df['Kode Wilayah'] = df['Kode Wilayah'].astype(int)

# Menghapus kolom 'WILAYAH'
df.drop('WILAYAH', axis=1, inplace=True)

# Menampilkan DataFrame untuk melihat hasilnya
print(df)

KeyError: 'WILAYAH'

In [37]:
# prompt: Change MONTH (Month from indonesia) to Month number from table

# Create a dictionary to map Indonesian month names to month numbers
month_mapping = {
    'Januari': 1,
    'Februari': 2,
    'Maret': 3,
    'April': 4,
    'Mei': 5,
    'Juni': 6,
    'Juli': 7,
    'Agustus': 8,
    'September': 9,
    'Oktober': 10,
    'November': 11,
    'Desember': 12
}

# Replace Indonesian month names with month numbers using the mapping
df['MONTH'] = df['MONTH'].map(month_mapping)
# # Menghapus jika Kode wilayah NA atau NaN
# # df = df.dropna(subset=['MONTH'])

# df['MONTH'] = df['MONTH'].astype(int)

print(df.head())


   BW  MONTH  YEAR  Kode Wilayah
0  10    NaN  2022             5
1  20    NaN  2022             5
2  10    NaN  2022             5
3  10    NaN  2022             5
4  20    NaN  2022             5
